In [4]:
n_iter=1

for i in range(n_iter):
  print(i)
  print(n_iter)
  print('hi')
  for hop in range(n_iter - i):
    print(hop)
    print(n_iter-i)
    print('hello')

0
1
hi
0
1
hello


In [ ]:
!pip install pyspark
!apt install openjdk-8-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 199 kB 20.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=fe5a669a9f795ba17e9cc88763a982c00d0db5433ac28cf2fc443c7212107265
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
E: Unable to locate package openjdk-8-headless


In [ ]:
import pyspark
from pyspark.sql import *
import pyspark.sql.functions as fn
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf

In [ ]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [ ]:
!pip install --target=$my_path findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark
findspark.init()

import pyspark
import numpy as np
import pandas as pd
import os

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import lit,col
from pyspark.sql import functions as F

In [ ]:
trainDf = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema = 'true').load(os.path.join('/content/drive/MyDrive/3-2 university/Big_data_process_and_application/input/heart.csv'))
trainDf.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)



In [ ]:
df= trainDf.withColumn('target1', trainDf['target'].cast('double')).drop('target')
df = df.drop('cp','thal','slope')
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- ca: integer (nullable = true)
 |-- target1: double (nullable = true)



In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed=11)

train = train.withColumn('testOrtrain', lit('train'))
test = test.withColumn('testOrtrain', lit('test'))

In [ ]:
df = (train.select('age', 'sex', 'trestbps','chol','fbs','restecg','thalach','exang','oldpeak','ca','target1','testOrtrain')).union(test.select('age','sex','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','ca','target1','testOrtrain'))
df.groupBy('testOrtrain').count().show()

+-----------+-----+
|testOrtrain|count|
+-----------+-----+
|      train|  218|
|       test|   85|
+-----------+-----+



In [ ]:
va = VectorAssembler(inputCols = ['age','sex','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','ca'], outputCol = 'features')

pipeline = Pipeline(stages = [va])
model = pipeline.fit(df)
myDf = model.transform(df)
myDf.show(5)

+---+---+--------+----+---+-------+-------+-----+-------+---+-------+-----------+--------------------+
|age|sex|trestbps|chol|fbs|restecg|thalach|exang|oldpeak| ca|target1|testOrtrain|            features|
+---+---+--------+----+---+-------+-------+-----+-------+---+-------+-----------+--------------------+
| 29|  1|     130| 204|  0|      0|    202|    0|    0.0|  0|    1.0|      train|(10,[0,1,2,3,6],[...|
| 34|  0|     118| 210|  0|      1|    192|    0|    0.7|  0|    1.0|      train|[34.0,0.0,118.0,2...|
| 34|  1|     118| 182|  0|      0|    174|    0|    0.0|  0|    1.0|      train|(10,[0,1,2,3,6],[...|
| 35|  0|     138| 183|  0|      1|    182|    0|    1.4|  0|    1.0|      train|[35.0,0.0,138.0,1...|
| 35|  1|     120| 198|  0|      1|    130|    1|    1.6|  0|    0.0|      train|[35.0,1.0,120.0,1...|
+---+---+--------+----+---+-------+-------+-----+-------+---+-------+-----------+--------------------+
only showing top 5 rows



In [ ]:
train = myDf.filter(myDf['testOrtrain'] == 'train')
trainDf, validateDf = train.randomSplit([0.7, 0.3], seed=11)
testDf = myDf.filter(myDf['testOrtrain'] == 'test')

In [ ]:
lr = (LogisticRegression().setLabelCol('target1').setFeaturesCol('features').setRegParam(0.0).setMaxIter(100).setElasticNetParam(0.))

In [ ]:
lrModel = lr.fit(trainDf)
lrDf = lrModel.transform(validateDf)
lrDf.groupBy('prediction','target1').count().show()

+----------+-------+-----+
|prediction|target1|count|
+----------+-------+-----+
|       1.0|    1.0|   27|
|       0.0|    1.0|    7|
|       1.0|    0.0|    4|
|       0.0|    0.0|   23|
+----------+-------+-----+



In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'target1')
print(evaluator.evaluate(lrDf)*100, '%')

82.29847494553375 %
